# Generic Rule Induction Notebook

Continuoulsy refined.

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [78]:

CONFIG1 = { 
    'NAME' : 'binary-churn-quantile-brcg', 
    'DATA_SET': '../data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'BRCG',
    'TARGET_LABEL': 'CHURN',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': 'T',
    'BASELINE': True
     }
CONFIG2 = { 
    'NAME' : 'continuous-churn', 
    'DATA_SET': '../data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'CHURN', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'pChurn',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': None,
    'BASELINE': True
     }
CONFIG3 = { 
    'NAME' : 'bike-demand', 
    'DATA_SET': '../data/SeoulBikeData.csv',
    'DATA_TYPES': {'Rented Bike Count': float, 'Hour': float, 'Humidity': float, 'Visibility (10m)': float, 'RatePlan': str},
    'DROP': ['Date'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Rented Bike Count',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Dew point temperature(C)',
    'POS_CLASS': None,
    'BASELINE': True
     }
CONFIG4 = { 
    'NAME' : 'heloc', 
    'DATA_SET': '../data/heloc.csv',
    'DATA_TYPES': {},
    'DROP': ['RiskPerformance'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Probabilities',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'ExternalRiskEstimate',
    'POS_CLASS': None,
    'BASELINE': True
     }
CONFIG5 = { 
    'NAME' : 'taiwan-credit', 
    'DATA_SET': '../data/TaiwanCreditData.csv',
    'DATA_TYPES': {},
    'DROP': ['DefaultNextMonth'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Probabilities',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Amount',
    'POS_CLASS': None,
    'BASELINE': True
     }
CONFIG6 = { 
    'NAME' : 'german-credit-brcg', 
    'DATA_SET': '../data/german_credit_codiert.csv',
    'DATA_TYPES': {},
    'DROP': ['Index'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'BRCG',
    'TARGET_LABEL': 'Target',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Credit Amount',
    'POS_CLASS': 0,
    'BASELINE': True
    
     }
CONFIG7 = { 
    'NAME' : 'german-credit-ripper', 
    'DATA_SET': '../data/german_credit_codiert.csv',
    'DATA_TYPES': {},
    'DROP': ['Index'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'NATIVE',
    'ALGO': 'RIPPER',
    'TARGET_LABEL': 'Target',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Credit Amount',
    'POS_CLASS': 1,
    'BASELINE': True
     }
CONFIG8 = { 
    'NAME' : 'binary-churn-ripper', 
    'DATA_SET': '../data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'RIPPER',
    'TARGET_LABEL': 'CHURN',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': 'T',
    'BASELINE': True
     }

CONFIG9 = { 
    'NAME' : 'compas-ripper', 
    'DATA_SET': '../data/compas.csv',
    'DATA_TYPES': {},
    'DROP': [],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'RIPPER',
    'TARGET_LABEL': 'recidivate-within-two-years',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'current-charge-degree',
    'POS_CLASS': 0,
    'BASELINE': True
     }

CONFIG = CONFIG8
print('Proceed with configuration:', CONFIG['NAME'])

Proceed with configuration: compas-ripper


In [79]:
import pandas as pd
import numpy as np
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREE':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper





# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [80]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG':
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1 
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7210 entries, 0 to 7209
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Unnamed: 0                   7210 non-null   int64 
 1   sex                          7210 non-null   object
 2   age                          7210 non-null   object
 3   juvenile-felonies            7210 non-null   object
 4   juvenile-misdemeanors        7210 non-null   object
 5   juvenile-crimes              7210 non-null   object
 6   priors                       7210 non-null   object
 7   current-charge-degree        7210 non-null   object
 8   recidivate-within-two-years  7210 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 507.1+ KB


,Unnamed: 0,sex,age,juvenile-felonies,juvenile-misdemeanors,juvenile-crimes,priors,current-charge-degree,recidivate-within-two-years
0,0,Female,18-20,=0,=0,=0,=0,Felony,1
1,1,Male,26-45,=0,=0,=0,2-3,Felony,1
2,2,Male,26-45,=0,=0,=0,=0,Misdemeanor,0
3,3,Male,23-25,=0,=0,=0,>3,Felony,0
4,4,Male,26-45,=0,=0,=0,>3,Felony,1


In [81]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

1    3960
0    3250
Name: recidivate-within-two-years, dtype: int64


### Train, Test Split

In [82]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (5047, 8) (5047,)
Test: (2163, 8) (2163,)


### Reference Performance

In [83]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [84]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=True) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5047 entries, 416 to 860
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype
---  ------                               --------------  -----
 0   (Unnamed: 0, <=, 740.6)              5047 non-null   int32
 1   (Unnamed: 0, <=, 1470.4)             5047 non-null   int32
 2   (Unnamed: 0, <=, 2184.000000000001)  5047 non-null   int32
 3   (Unnamed: 0, <=, 2915.4)             5047 non-null   int32
 4   (Unnamed: 0, <=, 3639.0)             5047 non-null   int32
 5   (Unnamed: 0, <=, 4339.4)             5047 non-null   int32
 6   (Unnamed: 0, <=, 5064.2)             5047 non-null   int32
 7   (Unnamed: 0, <=, 5768.8)             5047 non-null   int32
 8   (Unnamed: 0, <=, 6491.800000000001)  5047 non-null   int32
 9   (Unnamed: 0, >, 740.6)               5047 non-null   int32
 10  (Unnamed: 0, >, 1470.4)              5047 non-null   int32
 11  (Unnamed: 0, >, 2184.000000000001)   5047 non-null   in

operation,,not
value,,
416,0,1
4250,0,1
564,1,0
1224,1,0
142,0,1
1490,1,0
2399,0,1
4664,0,1
2004,1,0


### Rule Induction

In [85]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for BRCG
Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Iteration: 1, Objective: 0.3602
Iteration: 2, Objective: 0.3394
Iteration: 3, Objective: 0.3394
Iteration: 4, Objective: 0.3394
Training time: 2.990671396255493


### Evaluation

In [86]:
if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')


Accuracy: 0.6514100785945446
Balanced accuracy: 0.6387616609537032
Precision: 0.6708860759493671
Recall: 0.7427864798021434


In [87]:
#XGboost for Binary Classification
if CONFIG['TYPE'] == 'BINARY' and CONFIG['BASELINE'] == True:
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))

Accuracy: 0.6398520573277855
Balanced accuracy: 0.6295977784527271
Precision: 0.6671802773497689
Recall: 0.7139323990107173


In [88]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    print(estimator.rule_list_to_pretty_string())

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Number of rules: 1
['age != 18-20 AND juvenile-crimes not  AND priors != >3']
